In [86]:
import numpy as np
import pandas as pd
import sys
import os
import copy
import plotly.express as px
import importlib
import pickle
import math as m
import shutil
os.chdir('/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment')
import rmsd_functions as rmsd
importlib.reload(rmsd)

<module 'rmsd_functions' from '/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/rmsd_functions.py'>

In [118]:
pdb_dir = '/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/ORs_uncentered'
pdb_files = os.listdir(pdb_dir)
pdb_files = [pdb for pdb in pdb_files if ".pdb" in pdb]
data = {}
for file in pdb_files:
    name = file.split('.')[0]
    # for i in name.split('_'): # if there are multiple Olfr names for a single file, take the first 'Olfr' string
    #     if 'Olfr' in name: 
    #         name = name
    #         break
    name = name.split('-')[0]
    atom, coord, resid = rmsd.get_coordinates_pdb(f'{pdb_dir}/{file}')
    coord -= rmsd.centroid(coord)
    data[name] = {}
    data[name]["atom"] = atom
    data[name]["coord"] = coord
    data[name]["resid"] = resid


In [119]:
backbone = copy.deepcopy(data)
backbone = rmsd.get_backbone(backbone)


Reordered Olfr73_mOR atoms : ['C' 'CA' 'CB' 'CE' 'CG' 'N' 'O' 'SD' 'C' 'CA']
Reordered Olfr876 atoms : ['C' 'CA' 'CB' 'CE' 'CG' 'N' 'O' 'SD' 'C' 'CA']
Reordered Olfr983 atoms : ['C' 'CA' 'CB' 'CE' 'CG' 'N' 'O' 'SD' 'C' 'CA']
Reordered Olfr653 atoms : ['C' 'CA' 'CB' 'CE' 'CG' 'N' 'O' 'SD' 'C' 'CA']
Reordered Olfr168 atoms : ['C' 'CA' 'CB' 'CE' 'CG' 'N' 'O' 'SD' 'C' 'CA']
Backbone atoms extracted for  Olfr73_mOR
Backbone atoms extracted for  Olfr876
Backbone atoms extracted for  Olfr983
Backbone atoms extracted for  Olfr653
Backbone atoms extracted for  Olfr168


In [120]:
backbone = rmsd.trim_resid(backbone, start=25, end=285)

Resid trimed from 25 to 285 for Olfr73_mOR
Resid trimed from 25 to 285 for Olfr876
Resid trimed from 25 to 285 for Olfr983
Resid trimed from 25 to 285 for Olfr653
Resid trimed from 25 to 285 for Olfr168


In [121]:
small_Olfr = []
for Olfr in backbone:
    if backbone[Olfr]['resid'].max() != 285:
        small_Olfr.append(Olfr)
for Olfr in small_Olfr: 
    data.pop(Olfr)
    backbone.pop(Olfr)


In [122]:
# aligns the sequences in the input data by kabsch method
# Backbone is used to align, but rotation is asigned onto the full lengthed data 
data = rmsd.align_data(backbone, align="Olfr73_mOR", rotate_data=data)

        


Rotation calculated for Olfr73_mOR and applied to ROTATE DATA  Olfr876
Rotation calculated for Olfr73_mOR and applied to ROTATE DATA  Olfr983
Rotation calculated for Olfr73_mOR and applied to ROTATE DATA  Olfr653
Rotation calculated for Olfr73_mOR and applied to ROTATE DATA  Olfr168
RETURN new rotated data


In [83]:
# Center Olfr73 on its binding site
# Known binding site residues come from https://www.nature.com/articles/s42003-019-0384-8

f = '/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/AF_pdb/Olfr73_mOR-EG.pdb'
coord = rmsd.get_coordinates_pdb(f)

atomlist = [102, 105, 106, 109, 181, 182, 199, 203, 208, 259, 260, 273, 277, 280]
xatom = []
yatom = []
zatom = []
for i in range(len(coord[0])):

    if coord[2][i] in atomlist:
        xatom.append(coord[1][i][0])
        yatom.append(coord[1][i][1])
        zatom.append(coord[1][i][2])


geo_center = [np.mean(np.asarray(xatom)), 
              np.mean(np.asarray(yatom)), 
              np.mean(np.asarray(zatom))]


for i in range(len(xatom)):
    for j in range(len(xatom)):
        d = m.sqrt((xatom[i]-xatom[j]) ** 2 + (yatom[i]-yatom[j]) ** 2 + (zatom[i]-zatom[j])**2)
        maxd = max(maxd, d)


print(maxd)

# Found that max distance between atoms in voxel box is ≈ 30 Å, so voxel box should be kept to ≈ 32 Å


29.307503476072473


In [124]:
# Adjust center of each aligned protein by center of binding pocket on Olfr73
for k in data.keys():
    data[k]['coord'] -= geo_center

In [125]:
data_df = rmsd.dict_to_pdDataFrame(data)

In [126]:
# Save the aligned OR xyz coordinates for all AF pdb 
data_df.to_csv('/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/AF_pdb_df_test.csv')

In [127]:
data_df = pd.read_csv('/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/AF_pdb_df_test.csv', index_col=0)

In [128]:
rmsd.df_plot_all(data_df[data_df.models.isin(data_df['models'].unique()[:5])])

In [ ]:
# Isolate mouse gene names from .pdb directory

os.chdir('/home/users/jvs15/project-protein-fold/dual_cnn')
ors = pd.read_csv('odorants_ORs_paper.csv')
gene_names = ors['Gene'].unique()
mouse_genes = [x for x in gene_names if 'Olfr' in x]


mouse_pdbs = []
for filename in os.listdir('./OR_alignment/AF_pdb'):
    if 'Olfr' in filename:
        olfr = filename[0:len(filename)-4]
    mouse_pdbs.append(olfr)

print(mouse_pdbs)
    


In [90]:
# Generate a mapping between OR names from paper and their pdb files

# Extract unique gene names
os.chdir('/home/users/jvs15/project-protein-fold/dual_cnn')
ors = pd.read_csv('odorants_ORs_paper.csv')
gene_names = ors['Gene'].unique()

# Keys are gene names, values are .pdb filenames
d = {}
for gene in gene_names:
    olfr = gene.split('_')[0]

    # Loop through mouse genes
    if 'Olfr' in gene:
        # Check if exact mapping exists
        if any(gene == m for m in mouse_pdbs):
            d[gene] = f'{gene}.pdb'
            # shutil.copy(f'/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/AF_pdb/{d[gene]}', 
            #             f'/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/ORs_uncentered/{d[gene]}')

        # Account for unconventional names in dataset
        # AF makes filenames lowercase
        # TODO: Copy into ORs_uncentered
        elif any(olfr == m for m in mouse_pdbs):
            d[gene] = f'{gene.lower()}.pdb'

        # Account for unconventional names in pdb list
        elif any(olfr == m.split('_')[0] for m in mouse_pdbs):
            d[gene] = [f'{m}.pdb' for m in mouse_pdbs if olfr == m.split('_')[0]][0]
            shutil.copy(f'/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/AF_pdb/{d[gene]}', 
                        f'/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/ORs_uncentered/{d[gene]}')
        # This specific gene name has been changed (outlier)
        elif gene == 'Olfr174':
            d[gene] = 'Olfr175_Olfr175-ps1.pdb'
            shutil.copy(f'/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/AF_pdb/{d[gene]}', 
                        f'/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/ORs_uncentered/{d[gene]}')

    # Loop through human genes
    elif 'hOR' in gene:
        continue
    else:
        print(f'Invalid gene name {i} submitted')

print(len(os.listdir('./OR_alignment/ORs_uncentered')))



245


In [ ]:
# Add study data to odorants_ORs_paper.csv

In [54]:
# Find mouse pdb files that we already have

match = 0
mut = 0
for gene in mouse_genes:
    olfr = gene.split('_')[0]
    if any(gene == m for m in mouse_pdbs):
        match += 1
    elif any(olfr == m for m in mouse_pdbs):
        mut += 1
        print(gene)
    else:
        print(gene)
print(match, mut)



Olfr124_A183I
Olfr124_H176A
Olfr124_P182A
Olfr1362_F102Y
Olfr1362_I107L
Olfr1362_I204A
Olfr1362_S106A
Olfr1362_V207L
Olfr174
243 8
